#### We import all the libraries needed. In this case we are going to web scrapping by BeatifulSoup lib

In [2]:
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import csv, re, requests
import pandas as pd
import numpy as np
import urllib.request as urlreq
print("Libraries imported!")
#import matpotlib.plotlib as plt

Libraries imported!


First, we put the page in lxml format to read it.

In [3]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page=urlreq.urlopen(url)
soup = BeautifulSoup(page, "lxml")

We analize the data

In [67]:
##print(soup.prettify())

We check the code to find hte table with the data

In [4]:
table=soup.find('table', class_='wikitable sortable')
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

Now we extract the content of each data, we go through each row limited by "td" and then check if each row has 3 elements.

In [5]:
A=[]
B=[]
C=[]

for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
            A.append(cells[0].find(text=True))
            B.append(cells[1].find(text=True))
            C.append(cells[2].find(text=True))

Now, we create ahora dataframe form the lists.

In [61]:
toronto_df=pd.DataFrame(A,columns=['Postal Code'])
toronto_df['Borough']=B
toronto_df['Neighborhood']=C

I remove the 'Not assigned' rows

In [62]:
toronto_df=toronto_df[~toronto_df.Borough.str.contains("assigned")]
toronto_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


I group the data from Postal code joinning the Neighborhood

In [63]:
toronto_df=toronto_df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join).reset_index()
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge\n"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek\n"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


I replace Not assigned values from Neighborhood to the Borough values

In [65]:
toronto_df=toronto_df.replace('Not assigned',toronto_df['Borough'])
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge\n"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek\n"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


In [66]:
toronto_df.shape

(103, 3)

https://github.com/Maurochin/Coursera_Capstone